# 3D Temporal Analysis using Time

This section will including at least the mean, up to the fourth moment, of the time series mice data using two of the three principal components along with a 3rd axis for the time. Therefore, there will be every combination of two of the three PC's (3 graphs), all with a 3D axis with time. The moments will be calculated once per 24 hours (48 observations each day) with the intention of being able increase the amount of data points to more than 23, since experiment was taken over 23 days. 

The graphs will be analyzed to see if there is any discrepencies between certain principle components over time and if there is any emphasis on a certain PC over others.

In [45]:
import pandas as pd
from scipy import stats
import numpy as np

mouse_df_list = list()

# pull 16 mice PCA data
for i in range(1,17):
    df = pd.read_excel("data/newmicedata/PCFAnimal{}.xlsx".format(i))
    mouse_df_list.append(df)

pc_list = list()
# get rid of first 2 columns, left with 3 PC
# add dateTime back later or keep in the df's now?
for i in range(len(mouse_df_list)):
    pc_list.append(mouse_df_list[i].drop(['Unnamed: 0'], axis=1))

pc_list[0]

,DateTime,principal component 1,principal component 2,principal component 3
0,2022/11/04 14:01:00,17.132797,1.502005,-6.294937
1,2022/11/04 14:31:00,16.811791,2.291058,-7.172389
2,2022/11/04 15:01:00,15.028200,2.660764,-6.699671
3,2022/11/04 15:31:00,14.183293,2.758718,-5.704350
4,2022/11/04 16:01:00,11.949887,3.195511,-4.635048
...,...,...,...,...
1136,2022/11/28 06:01:00,-1.396782,-0.328081,-1.957727
1137,2022/11/28 06:31:00,-4.110406,0.404820,-0.490767
1138,2022/11/28 07:01:00,-0.687362,-0.372090,-2.771991
1139,2022/11/28 07:31:00,1.670424,-0.888519,-3.420816


In [46]:
# Create df's for each of the 3 different graphs
df_12 = []
df_13 = []
df_23 = []

for mouse in pc_list:
    df_12.append(mouse.drop(['principal component 3'], axis=1))
    df_13.append(mouse.drop(['principal component 2'], axis=1))
    df_23.append(mouse.drop(['principal component 1'], axis=1))

df_12[0]


,DateTime,principal component 1,principal component 2
0,2022/11/04 14:01:00,17.132797,1.502005
1,2022/11/04 14:31:00,16.811791,2.291058
2,2022/11/04 15:01:00,15.028200,2.660764
3,2022/11/04 15:31:00,14.183293,2.758718
4,2022/11/04 16:01:00,11.949887,3.195511
...,...,...,...
1136,2022/11/28 06:01:00,-1.396782,-0.328081
1137,2022/11/28 06:31:00,-4.110406,0.404820
1138,2022/11/28 07:01:00,-0.687362,-0.372090
1139,2022/11/28 07:31:00,1.670424,-0.888519


In [47]:
# 23 different points (24th is not a complete 24 hours)
DATAPOINTS = 23
NUMCOMPONENTS = 3

# these arrays contain arrays that represent curves, which contain 23 "points" each (arrays) which contain the 3 x y z coordinates
means = []
covs = []
skews = []
kurts = []
for mouse in df_12:
    # mouse_curve has 23 points in it
    mouse_curve_mean = []
    mouse_curve_cov = []
    mouse_curve_skew = []
    mouse_curve_kurt = []
    for i in range(1, DATAPOINTS + 1):
        mouse_mean = []
        mouse_cov = []
        mouse_skew = []
        mouse_kurt = []
        for j in range(1, NUMCOMPONENTS):
            # grab the first 24 rows of that column
            # mouse is mouse n's dataFrame
            column = (mouse[['principal component {}'.format(j)]].iloc[(i * 48) - 48:i * 48]).to_numpy()
            # column is an array of arrays with one number in each inner array so refactor to just one array with 24 elements
            column_refactor = []
            for n in range(len(column)):
                column_refactor.append(column[n][0])
            # run moment calculations
            # column = df.to_numpy()
            col_moments = stats.describe(column_refactor)
            mouse_mean.append((col_moments).mean)
            mouse_cov.append((col_moments).variance)
            mouse_skew.append((col_moments).skewness)
            mouse_kurt.append((col_moments).kurtosis)
        mouse_curve_mean.append(mouse_mean)
        mouse_curve_cov.append(mouse_cov)
        mouse_curve_skew.append(mouse_skew)
        mouse_curve_kurt.append(mouse_kurt)
    means.append(mouse_curve_mean)
    covs.append(mouse_curve_cov)
    skews.append(mouse_curve_skew)
    kurts.append(mouse_curve_kurt)

print(len(means))
mouse_curve_kurt

16


[[-1.4831479071135023, -1.2680000317616198],
 [-1.7515928371958633, -1.6230267785377341],
 [-1.7053042614712548, -1.613919463054078],
 [-1.3161894073791698, -1.3426586995339087],
 [-1.2492060891595984, -1.2741757053464264],
 [-0.9566888635611055, -1.3339393461814277],
 [-0.9939097755079511, -1.0958328488629059],
 [-1.445026518479188, -1.4258838449646383],
 [-1.4728408244328797, -1.383080255574382],
 [-0.7583497408452153, -0.9019986609114423],
 [-1.0293920529290244, -0.5053182004350356],
 [-0.9642282236470456, -0.8278896040130328],
 [-1.1970491560204526, -0.6583690483017959],
 [-0.9867013934972633, -0.8834623150905192],
 [-1.1453952126519975, -0.8004204103658759],
 [-1.1294420664434521, -1.1410041066618415],
 [-0.9363021470426296, -0.8370462207884022],
 [-0.7393272706523724, -0.9976162419834043],
 [-1.1259249478338067, -1.070991430112346],
 [-1.3414799442230139, -1.3200289845822624],
 [-1.0740693850526821, -0.958087909770085],
 [-1.2656097176321006, -1.3417252076444235],
 [-1.2101472470

In [48]:
# create times to add back to curve df lists
times = []
for j in range(1, DATAPOINTS + 1):
    date = (pc_list[0]['DateTime'].iloc[(j * 48) - 48])
    times.append(date)

# array of the curves of each of the 16 mice with PC 1 and 2
curves_df_12 = []
curves_df_13 = []
curves_df_23 = []
# add DateTime column back to every mouse mean
for mean in means:
    mean_df_12 = pd.DataFrame(mean, columns = ['PC_1', 'PC_2'])
    mean_df_12.insert(0, 'DateTime', times)
    curves_df_12.append(mean_df_12)

    mean_df_13 = pd.DataFrame(mean, columns = ['PC_1', 'PC_3'])
    mean_df_13.insert(0, 'DateTime', times)
    curves_df_13.append(mean_df_13)

    mean_df_23 = pd.DataFrame(mean, columns = ['PC_2', 'PC_3'])
    mean_df_23.insert(0, 'DateTime', times)
    curves_df_23.append(mean_df_23)

mean_df_13

,DateTime,PC_1,PC_3
0,2022/11/04 14:01:00,8.088046,0.045167
1,2022/11/05 14:01:00,5.763668,0.182189
2,2022/11/06 14:01:00,5.062267,0.097740
3,2022/11/07 14:01:00,3.513602,-0.198166
4,2022/11/08 14:01:00,3.496771,0.047351
5,2022/11/09 14:01:00,2.405407,-0.143135
6,2022/11/10 14:01:00,2.091982,-1.254267
7,2022/11/11 14:01:00,2.702987,-1.210891
8,2022/11/12 14:01:00,1.408727,-0.651537
9,2022/11/13 14:01:00,1.870238,-1.366102


# Mean with PC 1 & 2 Graph

In [49]:
import plotly.express as px
import plotly.graph_objects as go


# first plotting mean
fig = px.line_3d(curves_df_12[0], x='DateTime', y='PC_1', z='PC_2', title='PC 1&2 Plot')

for i in range(1, len(curves_df_12)):
    # Create additional line traces
    line_num = i + 1
    line_trace_n = go.Scatter3d(
        x=curves_df_12[i]['DateTime'].to_numpy(),
        y=curves_df_12[i]['PC_1'].to_numpy(),
        z=curves_df_12[i]['PC_2'].to_numpy(),
        mode='lines',
        name='Line {}'.format(line_num)
    )
    fig.add_trace(line_trace_n)


# Define the coordinates and labels for the points you want to label
point_x = []
point_y = []
point_z = []
for curve in curves_df_12:
    point_x.append(curve.at[0, 'DateTime'])
    point_y.append(curve.at[0, 'PC_1'])
    point_z.append(curve.at[0, 'PC_2'])

point_labels = ['1S', '2S', '3S', '4S', '5S', '6S', '7S', '8S', '9S', '10S',  '11S', '12S', '13S', '14S', '15S', '16S']

# Create a scatter trace with text annotations for the labeled points
scatter_trace = go.Scatter3d(
    x=point_x,
    y=point_y,
    z=point_z,
    mode='markers+text',
    marker=dict(
        size=5,
        color='red',
    ),
    text=point_labels,
    textposition='bottom center',
    name='Labeled Points'
)

# Add the scatter trace to the figure
fig.add_trace(scatter_trace)

# fig.write_html('downloads/PC12_plot.html')
fig

# Mean with PC 1 & 3 Graph

In [50]:
# plotting mean, pc 1 and 3
fig = px.line_3d(curves_df_13[0], x='DateTime', y='PC_1', z='PC_3', title='PC 1&3 Plot')

for i in range(1, len(curves_df_13)):
    # Create additional line traces
    line_num = i + 1
    line_trace_n = go.Scatter3d(
        x=curves_df_13[i]['DateTime'].to_numpy(),
        y=curves_df_13[i]['PC_1'].to_numpy(),
        z=curves_df_13[i]['PC_3'].to_numpy(),
        mode='lines',
        name='Line {}'.format(line_num)
    )
    fig.add_trace(line_trace_n)


# Define the coordinates and labels for the points you want to label
point_x = []
point_y = []
point_z = []
for curve in curves_df_13:
    point_x.append(curve.at[0, 'DateTime'])
    point_y.append(curve.at[0, 'PC_1'])
    point_z.append(curve.at[0, 'PC_3'])

point_labels = ['1S', '2S', '3S', '4S', '5S', '6S', '7S', '8S', '9S', '10S',  '11S', '12S', '13S', '14S', '15S', '16S']

# Create a scatter trace with text annotations for the labeled points
scatter_trace = go.Scatter3d(
    x=point_x,
    y=point_y,
    z=point_z,
    mode='markers+text',
    marker=dict(
        size=5,
        color='red',
    ),
    text=point_labels,
    textposition='bottom center',
    name='Labeled Points'
)

# Add the scatter trace to the figure
fig.add_trace(scatter_trace)

# fig.write_html('downloads/PC13_plot.html')
fig

# Mean with PC 2 & 3 Graph

In [51]:
# plotting mean, pc 2 and 3
fig = px.line_3d(curves_df_23[0], x='DateTime', y='PC_2', z='PC_3', title='PC 2&3 Plot')

for i in range(1, len(curves_df_23)):
    # Create additional line traces
    line_num = i + 1
    line_trace_n = go.Scatter3d(
        x=curves_df_23[i]['DateTime'].to_numpy(),
        y=curves_df_23[i]['PC_2'].to_numpy(),
        z=curves_df_23[i]['PC_3'].to_numpy(),
        mode='lines',
        name='Line {}'.format(line_num)
    )
    fig.add_trace(line_trace_n)


# Define the coordinates and labels for the points you want to label
point_x = []
point_y = []
point_z = []
for curve in curves_df_23:
    point_x.append(curve.at[0, 'DateTime'])
    point_y.append(curve.at[0, 'PC_2'])
    point_z.append(curve.at[0, 'PC_3'])

point_labels = ['1S', '2S', '3S', '4S', '5S', '6S', '7S', '8S', '9S', '10S',  '11S', '12S', '13S', '14S', '15S', '16S']

# Create a scatter trace with text annotations for the labeled points
scatter_trace = go.Scatter3d(
    x=point_x,
    y=point_y,
    z=point_z,
    mode='markers+text',
    marker=dict(
        size=5,
        color='red',
    ),
    text=point_labels,
    textposition='bottom center',
    name='Labeled Points'
)

# Add the scatter trace to the figure
fig.add_trace(scatter_trace)

# fig.write_html('downloads/PC23_plot.html')
fig